In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Pyspark

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=9d82114fd40423952e6bac6a7125ab737d84f7465ed00cea4336c2d623a6c4d9
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Import module

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
import pyspark.sql.functions as f

In [4]:
spark = SparkSession.builder.appName("Recommendation").getOrCreate()

In [7]:
file_path = '/content/drive/MyDrive/Data_project/ratings_Beauty.csv'
df_buauty = spark.read.format('csv').option('header',True).option('inferSchema',True).load(file_path) #data frame, header will infer column types from csv
df_buauty.show()

+--------------+----------+------+----------+
|        UserId| ProductId|Rating| Timestamp|
+--------------+----------+------+----------+
|A39HTATAQ9V7YF|0205616461|   5.0|1369699200|
|A3JM6GV9MNOF9X|0558925278|   3.0|1355443200|
|A1Z513UWSAAO0F|0558925278|   5.0|1404691200|
|A1WMRR494NWEWV|0733001998|   4.0|1382572800|
|A3IAAVS479H7M7|0737104473|   1.0|1274227200|
| AKJHHD5VEH7VG|0762451459|   5.0|1404518400|
|A1BG8QW55XHN6U|1304139212|   5.0|1371945600|
|A22VW0P4VZHDE3|1304139220|   5.0|1373068800|
|A3V3RE4132GKRO|130414089X|   5.0|1401840000|
|A327B0I7CYTEJC|130414643X|   4.0|1389052800|
|A1BG8QW55XHN6U|130414643X|   5.0|1372032000|
| AIFAAVTUYHEHB|130414643X|   4.0|1378252800|
| AVOGV98AYOFG2|1304146537|   5.0|1372118400|
|A22VW0P4VZHDE3|130414674X|   5.0|1371686400|
| AVOGV98AYOFG2|1304168522|   5.0|1372118400|
| A6R426V4J7AOM|1304168522|   5.0|1373414400|
|A22VW0P4VZHDE3|1304174778|   5.0|1372896000|
| AKGB62WGF35J8|1304174778|   5.0|1372896000|
|A22VW0P4VZHDE3|1304174867|   5.0|

In [8]:
df_buauty = df_buauty.drop('Timestamp')
df_buauty.show()

+--------------+----------+------+
|        UserId| ProductId|Rating|
+--------------+----------+------+
|A39HTATAQ9V7YF|0205616461|   5.0|
|A3JM6GV9MNOF9X|0558925278|   3.0|
|A1Z513UWSAAO0F|0558925278|   5.0|
|A1WMRR494NWEWV|0733001998|   4.0|
|A3IAAVS479H7M7|0737104473|   1.0|
| AKJHHD5VEH7VG|0762451459|   5.0|
|A1BG8QW55XHN6U|1304139212|   5.0|
|A22VW0P4VZHDE3|1304139220|   5.0|
|A3V3RE4132GKRO|130414089X|   5.0|
|A327B0I7CYTEJC|130414643X|   4.0|
|A1BG8QW55XHN6U|130414643X|   5.0|
| AIFAAVTUYHEHB|130414643X|   4.0|
| AVOGV98AYOFG2|1304146537|   5.0|
|A22VW0P4VZHDE3|130414674X|   5.0|
| AVOGV98AYOFG2|1304168522|   5.0|
| A6R426V4J7AOM|1304168522|   5.0|
|A22VW0P4VZHDE3|1304174778|   5.0|
| AKGB62WGF35J8|1304174778|   5.0|
|A22VW0P4VZHDE3|1304174867|   5.0|
|A1BG8QW55XHN6U|1304174867|   5.0|
+--------------+----------+------+
only showing top 20 rows



In [10]:

na_counts = df_buauty.select([f.sum(f.col(c).isNull().cast("int")).alias(c) for c in df_buauty.columns])
na_counts.show()

+------+---------+------+
|UserId|ProductId|Rating|
+------+---------+------+
|     0|        0|     0|
+------+---------+------+



In [11]:
rows = df_buauty.count()
cols = len(df_buauty.columns)
print(rows,cols)

2023070 3


* let us understand how freaquently user has buy the product

In [12]:
df_beuaty_agg = df_buauty.select('UserId', 'ProductId').groupby('UserId', 'ProductId').agg(count('*').alias('count')).orderBy('UserId')
df_beuaty_agg.show()

+--------------------+----------+-----+
|              UserId| ProductId|count|
+--------------------+----------+-----+
|A00008821J0F472ND...|B007T8XPC6|    1|
|A000186437REL8X2R...|B000JD4N9M|    1|
|A0002574WYJMBWKNCPY8|B00AH4TJNS|    1|
|A00029263J863WSR0...|B00766SGE8|    1|
|A00031961JI1CBNV98TW|B002OO16QC|    1|
|A000325234LCBTFVL...|B006ZN1B1S|    1|
|A0003526DBE3EVU1DXE5|B00F1AITPM|    1|
|A0004690232S2SX49...|B009L0YD6M|    1|
|A000622834IP6B056...|B00AE078TM|    1|
|A0007094NAUEOIOA53YH|B005UFN3VS|    1|
|A00088021N0VU5Z2U...|B003AMEUS8|    1|
|A0010876CNE3ILIM9HV0|B001KYRVSO|    1|
|A0010876CNE3ILIM9HV0|B004MKEMPI|    1|
|A0010876CNE3ILIM9HV0|B005HKST1U|    1|
|A0010876CNE3ILIM9HV0|B0055MYJ0U|    1|
|A0011102257KBXODK...|B009PZGYJM|    1|
|A001114613O3F18Q5...|B007HSGKXY|    1|
|A00116502XCN8ZW09...|B0024WDWU2|    1|
|A00120381FL204MYH...|B006SVCY6I|    1|
|A001235832OWO8HZG...|B001D5CG9E|    1|
+--------------------+----------+-----+
only showing top 20 rows



In [20]:
df_buauty.groupBy('UserID','ProductID').count().orderBy(desc('count')).show()

+--------------+----------+-----+
|        UserID| ProductID|count|
+--------------+----------+-----+
|A2H3NOCLLEPOKQ|7535842801|    1|
|A1RGXKJ4RFIOCX|1304624544|    1|
|A3G6XNM240RMWA|7806397051|    1|
|A3VUS3HC1CCVOQ|5357955948|    1|
| AF508G9N8T5OY|9746427962|    1|
|A2LRQ8GV6PZMHX|7806397051|    1|
|A2EDV2NPKN69YM|9788080626|    1|
|A264BXIV14DIWA|9511181564|    1|
|A2L8AWOIWML9CH|9790772238|    1|
|A3KK59I1TY3MRB|9788071171|    1|
| ACUHBB2EEXH24|9790777450|    1|
| A2PZOIS5NOHEA|9788071260|    1|
| AXQDDGADNZNNG|9790789556|    1|
| AYB66VHHXKRQQ|9790778872|    1|
|A3N5L80QA6SDZ2|9790795416|    1|
|A27JHZ3V0NYS2C|9790799365|    1|
|A2GWFNKXW7REGE|9790801491|    1|
|A15SQDXX0VZPY2|9790801076|    1|
|A29LEQ3L7XN2BF|B00000JGVX|    1|
|A12L11Y59Y8Z54|B00004TMFE|    1|
+--------------+----------+-----+
only showing top 20 rows



* it can be understand that no user has buy same product more than that

In [13]:
rows = df_beuaty_agg.count()
cols = len(df_beuaty_agg.columns)
print(rows,cols)

2023070 3


In [27]:
#we want to decrement our row size, its almost 10 million, lets reduce it inorder to process our data much faster
df_beuaty_agg = df_beuaty_agg.limit(20000)

*  Let's convert the user id and product id columns into unique integers

In [28]:
old_strindexer = [StringIndexer(inputCol = col, outputCol = col + '_index').fit(df_beuaty_agg) for col in list(set(df_beuaty_agg.columns)- set(['count']))]
indexer = [curr_strindexer.setHandleInvalid("keep") for curr_strindexer in old_strindexer]

pipeline = Pipeline(stages = indexer)

data = pipeline.fit(df_beuaty_agg).transform(df_beuaty_agg)
data.show()

+--------------------+----------+-----+------------+---------------+
|              UserId| ProductId|count|UserId_index|ProductId_index|
+--------------------+----------+-----+------------+---------------+
|A00008821J0F472ND...|B007T8XPC6|    1|      3143.0|        11632.0|
|A000186437REL8X2R...|B000JD4N9M|    1|      3144.0|          637.0|
|A0002574WYJMBWKNCPY8|B00AH4TJNS|    1|      3145.0|        12802.0|
|A00029263J863WSR0...|B00766SGE8|    1|      3146.0|        11283.0|
|A00031961JI1CBNV98TW|B002OO16QC|    1|      3147.0|         7532.0|
|A000325234LCBTFVL...|B006ZN1B1S|    1|      3148.0|          973.0|
|A0003526DBE3EVU1DXE5|B00F1AITPM|    1|      3149.0|        13863.0|
|A0004690232S2SX49...|B009L0YD6M|    1|      3150.0|        12489.0|
|A000622834IP6B056...|B00AE078TM|    1|      3151.0|         2626.0|
|A0007094NAUEOIOA53YH|B005UFN3VS|    1|      3152.0|         2331.0|
|A00088021N0VU5Z2U...|B003AMEUS8|    1|      3153.0|          833.0|
|A0010876CNE3ILIM9HV0|B001KYRVSO| 

In [29]:
data = data.select('UserId_index', 'ProductId_index', 'count').orderBy('UserId_index')
data.show()

+------------+---------------+-----+
|UserId_index|ProductId_index|count|
+------------+---------------+-----+
|         0.0|         6230.0|    1|
|         0.0|        14214.0|    1|
|         0.0|           30.0|    1|
|         0.0|         1591.0|    1|
|         0.0|         6237.0|    1|
|         0.0|         1178.0|    1|
|         0.0|         7140.0|    1|
|         0.0|         4193.0|    1|
|         0.0|         1619.0|    1|
|         0.0|         8222.0|    1|
|         0.0|          727.0|    1|
|         0.0|         3101.0|    1|
|         0.0|          337.0|    1|
|         0.0|         1912.0|    1|
|         0.0|         7369.0|    1|
|         0.0|         3224.0|    1|
|         0.0|         1566.0|    1|
|         0.0|         8184.0|    1|
|         0.0|         3522.0|    1|
|         0.0|            8.0|    1|
+------------+---------------+-----+
only showing top 20 rows



In [30]:
(training, test) = data.randomSplit([0.8,0.2])

In [31]:
USERID = "UserId_index"
PRODUCT = "ProductId_index"
COUNT = "count"

als = ALS(maxIter = 5, regParam = 0.01, userCol = USERID, itemCol = PRODUCT, ratingCol = COUNT)
# als.checkpointInterval = 2
print(als)
model = als.fit(training)

predictions = model.transform(test)

ALS_90cadb781cd5


In [32]:
recs = model.recommendForAllUsers(10)
recs.show()
recs.take(1)

+------------+--------------------+
|UserId_index|     recommendations|
+------------+--------------------+
|           1|[{176, 1.6942188}...|
|           3|[{139, 1.8378881}...|
|           5|[{25, 1.690864}, ...|
|           6|[{34, 1.4461343},...|
|           9|[{25, 1.5553919},...|
|          12|[{35, 1.7730621},...|
|          13|[{495, 1.4527651}...|
|          15|[{170, 1.9861124}...|
|          16|[{13, 1.7804167},...|
|          17|[{58, 1.6713225},...|
|          19|[{352, 1.556861},...|
|          20|[{67, 1.9254}, {1...|
|          22|[{60, 1.5218568},...|
|          26|[{289, 1.7522482}...|
|          27|[{122, 1.698113},...|
|          28|[{122, 1.633249},...|
|          31|[{466, 1.485825},...|
|          34|[{630, 1.4097846}...|
|          35|[{73, 1.7835364},...|
|          37|[{69, 1.459546}, ...|
+------------+--------------------+
only showing top 20 rows



[Row(UserId_index=1, recommendations=[Row(ProductId_index=176, rating=1.6942187547683716), Row(ProductId_index=109, rating=1.6723308563232422), Row(ProductId_index=22, rating=1.5146303176879883), Row(ProductId_index=220, rating=1.5122601985931396), Row(ProductId_index=379, rating=1.4668844938278198), Row(ProductId_index=143, rating=1.399141788482666), Row(ProductId_index=141, rating=1.3452752828598022), Row(ProductId_index=243, rating=1.3073979616165161), Row(ProductId_index=2669, rating=1.2915350198745728), Row(ProductId_index=67, rating=1.281299114227295)])]